In [1]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from repeater import retry
import json


In [15]:
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "sejong"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"sejong":  {"iconservice": "https://sejong.net.solidwallet.io", "nid":83},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]
NONCE = 100
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

# checking transaction result
def check_tx_result(res):
    if res['status'] == 0:
        print("Tx Failed, Message: " + res['failure']['message'])
    else:
        print("Success")
        
# Set sejong testnet address as required
GOVERNANCE_SCORE = "cx06eaddea58321a98bd0e1c9bd70f4680d4f3a73a"
DEX_SCORE = "cx113466143604608cbf58d84674d9360b29f08530"
LOANS_SCORE = "cx2148bdb92054b2306076f3579c0ab7fced91a9ca"
REWARDS_SCORE = ""
DIVIDENDS_SCORE = ""

#MAINNET ADDRESS
# GOVERNANCE_SCORE = "cx44250a12074799e26fdeee75648ae47e2cc84219"
# DEX_SCORE = "cxa0af3165c08318e988cb30993b3048335b94af6c"
# LOANS_SCORE = "cx66d4d90f5f113eba575bf793570135f9b10cece1"
# REWARDS_SCORE = "cx10d59e8103ab44635190bd4139dbfd682fa2d07e"
# DIVIDENDS_SCORE = ""


In [7]:
# Load the wallet using keystore.
with open("./keystore/staking_test.pwd", "r") as f:
    key_data = f.read()
deployer_wallet = KeyWallet.load("./keystore/staking_test.json", key_data)

#using private key to load test wallet to perform transactions in case of need
# tester_wallet = KeyWallet.load(bytes.fromhex(""))
# tester_wallet.get_address()

FileNotFoundError: [Errno 2] No such file or directory: './keystore/staking_test.pwd'

In [8]:
# function to deploy contract.
def deploy_contract(wallet, _to_address, _name, params):
    deploy_transaction = DeployTransactionBuilder() \
        .from_(wallet) \
        .to(_to_address) \
        .nid(NID) \
        .params(params) \
        .version(3) \
        .nonce(100) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(_name)) \
        .build()

    estimate_step = icon_service.estimate_step(deploy_transaction)
    step_limit = estimate_step + 40000000
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

    return icon_service.send_transaction(signed_transaction)

# Function to send tx in the network
def send_tx(wallet, _to_address, method, params, value):
    transaction = CallTransactionBuilder() \
        .from_(wallet.get_address()) \
        .to(_to_address) \
        .value(value) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(method) \
        .params(params) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    return get_tx_result(tx_hash)

# Function to call read-only functions.
def call_tx(_to_address, method, params):
    call = CallBuilder() \
        .to(_to_address) \
        .method(method) \
        .params(params) \
        .build()

    return icon_service.call(call)

#swap function for DEX
def swap(from_token, to_token, amount,wallet):
    data2 = json.dumps({"method": "_swap", "params": {"toToken":str(to_token)}})
    params = {'_to': contract_addresses['dex'], '_value': amount, '_data': data2.encode()}
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(from_token)\
        .step_limit(10000000)\
        .value(0)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    return get_tx_result(tx_hash)

## Deploy Staked LP contract

In [9]:
#Governance score should be provided in on_install.
res = get_tx_result(deploy_contract(deployer_wallet.get_address(), GOVERNANCE_ADDRESS, './core_contracts/stakedLp',
                                   {"_governance": GOVERNANCE_SCORE}))
print('scoreAddress:', res.get('scoreAddress', ''))
stakedlp_score = res.get('scoreAddress', '')
check_tx_result(res)

NameError: name 'deployer_wallet' is not defined

## setAdmin for stakedLp

In [16]:
# Governance contract is set as admin.

res = send_tx(deployer_wallet, stakedlp_score, "setAdmin", {"_admin": GOVERNANCE_SCORE}, 0)
check_tx_result(res)

NameError: name 'deployer_wallet' is not defined

## Set StakedLp address in governance

In [11]:
# Getting all contract addresses from balanced governance score
contract_addresses = call_tx(GOVERNANCE_SCORE, 'getAddresses', {})

#Add stakedLp address to the dictdb and sets the addresses to the Balanced Governance contract.
contract_addresses['stakedLp'] = stakedlp_score
res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setAddresses", {"_addresses": contract_addresses}, 0)
check_tx_result(res)

# Now the read-only fn will have stakedLp address too
print(call_tx(GOVERNANCE_SCORE, 'getAddresses', {}))

JSONRPCException: {'code': -30002, 'message': 'SystemError(-30002) : E0002:ContractNotFound'}

## SetDex and SetRewards on stakedLp

In [19]:
# setDex address on stakedLp contract.
res = send_tx(deployer_wallet, stakedlp_score, "setDex", {"_dex": DEX_SCORE}, 0)
check_tx_result(res)

# setRewards address on stakedLp contract.
res = send_tx(deployer_wallet, stakedlp_score, "setRewards", {"_address": REWARDS_SCORE}, 0)
check_tx_result(res)

NameError: name 'deployer_wallet' is not defined

## Update Loans

In [13]:
# We need to set continuous_rewards_day in on_update.
# vardb to set is _continuous_reward_day.
#eg :self._continuous_reward_day.set(45).
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), LOANS_SCORE, './core_contracts/loans', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)

NameError: name 'deployer_wallet' is not defined

## Update Rewards

In [ ]:
# We need to set continuous_rewards_day in on_update.
# vardb to set is _continuous_rewards_day.
#eg :self._continuous_rewards_day.set(45).

tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), REWARDS_SCORE, './core_contracts/rewards', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)

## Update Dividends

In [ ]:
# We need to set setDividendsOnlyToStakedBalnDay in dividends score.
# First we need to update the dividends score and call the setter method to set _dividends_enabled_to_staked_baln_day
# from governance.
# Step: 1

tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), DIVIDENDS_SCORE, './core_contracts/dividends', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)


In [ ]:
# Step: 2
# setDividendsOnlyToStakedBalnDay on dividends contract.
res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setDividendsOnlyToStakedBalnDay", {"_day": }, 0)
check_tx_result(res)